<a href="https://colab.research.google.com/github/Strata-Tech/Covid19/blob/main/covid19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip3 install plotly==4.8

     |████████████████████████████████| 11.5 MB 16.0 MB/s 
  Attempting uninstall: plotly
    Found existing installation: plotly 4.4.1
    Uninstalling plotly-4.4.1:
      Successfully uninstalled plotly-4.4.1


In [1]:
# import the relevant packages
import os
import pandas as pd
import numpy as np

# ploty is an interactive visualization package built using javascript. 
# it is mostly used for explantory visualization (presenting to others)
import plotly.express as px

# requests is a package that allows Python to talk to the world wide web (www)
import requests

In [2]:
#Using the data provided by John Hopkins at https://github.com/CSSEGISandData/COVID-19
#datasets used are confirmed,deaths and recovered cases.
COVID_DATA_URLS = [
  'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv',                 
  'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv',
  'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv'              
]

In [3]:
#to check if links are working

for url in COVID_DATA_URLS:
  response=requests.head(url)
  print(str(response.status_code)+':'+response.request.url)

200:https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv
200:https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv
200:https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv


In [4]:
#Creating a function which downloads file and save it in colab hosted runtime if file does not exist. If file exists, reuse the saved file


url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'


os.listdir()

['.config',
 'time_series_covid19_deaths_global.csv',
 'time_series_covid19_recovered_global.csv',
 'time_series_covid19_confirmed_global.csv',
 'sample_data']

In [5]:
def load_df_from_url(url):
  filename=url.split('/')[-1]
  if filename in os.listdir():
    df=pd.read_csv(filename)
    print(f"Using saved file {filename} on colab hosted runtime")

  else:
    df=pd.read_csv(url)
    df.to_csv(f"{filename}",index=False)
    print(f"Downloaded file {filename} from internet and saved to colab hosted runtime")

  return df, filename
  


In [6]:
#testing function with first url in the list

url = COVID_DATA_URLS[0]
df, filename = load_df_from_url(url)

#to see all columns
pd.options.display.width=9999
pd.options.display.max_columns=15

print(df.head())
print(df.tail())


Using saved file time_series_covid19_confirmed_global.csv on colab hosted runtime
  Province/State Country/Region       Lat       Long  1/22/20  1/23/20  1/24/20  ...  8/1/21  8/2/21  8/3/21  8/4/21  8/5/21  8/6/21  8/7/21
0            NaN    Afghanistan  33.93911  67.709953        0        0        0  ...  146523  147985  148572  148933  149361  149810  149810
1            NaN        Albania  41.15330  20.168300        0        0        0  ...  133121  133146  133211  133310  133442  133591  133730
2            NaN        Algeria  28.03390   1.659600        0        0        0  ...  172564  173922  175229  176724  178013  179216  180356
3            NaN        Andorra  42.50630   1.521800        0        0        0  ...   14678   14747   14766   14797   14809   14836   14836
4            NaN         Angola -11.20270  17.873900        0        0        0  ...   42815   42970   43070   43158   43269   43487   43592

[5 rows x 568 columns]
    Province/State      Country/Region        La

In [ ]:
#writing a loop for each datafile in the list 
#so that we can load the file in memory
#rename the columns from Province/State to Province,Country/Region to Country
#change all columns to lower case
#Fill missing values in province with values from country
#Check and drop columns with overwhelming null values.
#Convert the datset into a panel set with dates and countries as rows.
#Merge the panel dataframe into the main dataset, to arrive at confirmed, death and recovered columns

In [8]:
# ## For each data file,
# # enumerate creates a tuple with a counter that starts from 0 and the elements of the list, e.g. (0, URL1), (1, URL2), etc.
# for i, url in enumerate(COVID_DATA_URLS):

#     # Load the file into memory
#     df, filename = load_df_from_url(url)

#     # rename columns
#     df.rename(columns={'Province/State': 'Province', 'Country/Region':'Country'}, inplace=True)
#     # and make the column names lowercase, 
#     # general method, can be used in all dfs
#     df.columns = [c.lower() for c in df.columns]

#     # fill missing provinces/states with the country/region name
#     df['province'].fillna(df['country'], inplace=True)

#     # Create a pivot table with 'Province/State', 'Country/Region', 'Lat', 'Long' groups
#     df = df.groupby(['province','country', 'lat', 'long']).sum().reset_index()

#     # split the filename to get the keywords we want (confirmed, deaths, recovered)
#     value_name = filename.split('_')[-2]

#     # pivot the dataframe into a panel format.
#     df = df.melt(id_vars=['province','country', 'lat', 'long'], var_name='date',value_name=value_name)

#     # convert date column to datetime
#     df['date'] = pd.to_datetime(df['date'])

#     # Merge the dataframe into the main dataset
#     # checking if it's the first df to be read
#     # i should be equal to 0
#     # i is created from enumerate()
#     if i == 0:
#         dataset = df
#     # print(list(df))
#     else:
#         dataset = pd.merge(dataset, df, how='outer', on=['province','country', 'date', 'lat', 'long'])

# dataset.sort_values(['province','country', 'date'], inplace=True)


Using saved file time_series_covid19_confirmed_global.csv on colab hosted runtime
Using saved file time_series_covid19_deaths_global.csv on colab hosted runtime
Using saved file time_series_covid19_recovered_global.csv on colab hosted runtime


In [16]:
#writing a loop for each datafile in the list 
#so that we can load the file in memory
#rename the columns from Province/State to Province,Country/Region to Country
#change all columns to lower case
#Fill missing values in province with values from country
#Check and drop columns with overwhelming null values.
#Convert the datset into a panel set with dates and countries as rows.
#Merge the panel dataframe into the main dataset, to arrive at confirmed, death and recovered columns

for i, url in enumerate(COVID_DATA_URLS):

  #load file
  df,filename=load_df_from_url(url)
  

  #rename columns
  df.rename(columns={'Province/State':'Province','Country/Region':'Country'},inplace=True)


  
  #Change columns to lower case
  df.columns=[col.lower() for col in df.columns]


  #Fill missing values in province with values from country 
  df['province'].fillna(df['country'],inplace=True)
  

  # Create a pivot table with 'Province/State', 'Country/Region', 'Lat', 'Long' groups
  df=df.groupby(['province','country','lat','long']).sum().reset_index()
 


   # split the filename to get the keywords we want (confirmed, deaths, recovered)
  value_name=filename.split('_')[-2]
 

  # # pivot the dataframe into a panel format.
  df=df.melt(id_vars=['province','country','lat','long'],var_name='date',value_name=value_name)
 

  

  #convert date to datetime
  df['date']=pd.to_datetime(df['date'])
  

  # Merge the dataframe into the main dataset
  # checking if it's the first df to be read
  # i should be equal to 0
  # i is created from enumerate()

  if i==0:
    dataset=df

  else:
    dataset=pd.merge(dataset,df,how='outer',on=['province','country','lat','long','date'])



dataset.sort_values(['province','country','lat','long'],inplace=True)



Using saved file time_series_covid19_confirmed_global.csv on colab hosted runtime
Using saved file time_series_covid19_deaths_global.csv on colab hosted runtime
Using saved file time_series_covid19_recovered_global.csv on colab hosted runtime


In [17]:
dataset.head()

,province,country,lat,long,date,confirmed,deaths,recovered
0,Afghanistan,Afghanistan,33.93911,67.709953,2020-01-22,0.0,0.0,0.0
277,Afghanistan,Afghanistan,33.93911,67.709953,2020-01-23,0.0,0.0,0.0
554,Afghanistan,Afghanistan,33.93911,67.709953,2020-01-24,0.0,0.0,0.0
831,Afghanistan,Afghanistan,33.93911,67.709953,2020-01-25,0.0,0.0,0.0
1108,Afghanistan,Afghanistan,33.93911,67.709953,2020-01-26,0.0,0.0,0.0


In [21]:
# check if all the status rows are filled, return those rows with missing data
# isnull() returns any rows that has null values

null_data=dataset[dataset[['confirmed','deaths','recovered']].isnull().any(axis=1)]
null_data

,province,country,lat,long,date,confirmed,deaths,recovered
2,Alberta,Canada,53.9333,-116.5765,2020-01-22,0.0,0.0,NaN
279,Alberta,Canada,53.9333,-116.5765,2020-01-23,0.0,0.0,NaN
556,Alberta,Canada,53.9333,-116.5765,2020-01-24,0.0,0.0,NaN
833,Alberta,Canada,53.9333,-116.5765,2020-01-25,0.0,0.0,NaN
1110,Alberta,Canada,53.9333,-116.5765,2020-01-26,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...
155115,Yukon,Canada,64.2823,-135.0000,2021-08-03,610.0,8.0,NaN
155392,Yukon,Canada,64.2823,-135.0000,2021-08-04,616.0,8.0,NaN
155669,Yukon,Canada,64.2823,-135.0000,2021-08-05,623.0,8.0,NaN
155946,Yukon,Canada,64.2823,-135.0000,2021-08-06,624.0,8.0,NaN


In [23]:
# set maximum number of rows to display to be all
pd.options.display.max_rows=30

# groupby the key columns (except date) and look at their mean to discover what's wrong
null_countries=null_data.groupby(['province','country','lat','long']).mean().reset_index()
null_countries.sort_values(['country','province'])

# Canada reports confirmed and deaths by province, but recovered by entire country sum
# Hebei, Henan, Mozambique's Recovered lat long are smaller than Confirmed,Deaths lat long
# Syria's, Timor-Leste's Recovered lat long are larger than Confirmed,Deaths lat long


,province,country,lat,long,confirmed,deaths,recovered
0,Alberta,Canada,53.933300,-116.576500,80029.567376,908.264184,NaN
1,British Columbia,Canada,53.726700,-127.647600,48835.008865,703.329787,NaN
2,Canada,Canada,56.130400,-106.346800,NaN,NaN,468525.475177
3,Diamond Princess,Canada,0.000000,0.000000,0.056738,0.893617,NaN
4,Grand Princess,Canada,0.000000,0.000000,11.712766,0.000000,NaN
9,Manitoba,Canada,53.760900,-98.813900,18112.093972,421.544326,NaN
12,New Brunswick,Canada,46.565300,-66.461900,808.148936,14.219858,NaN
13,Newfoundland and Labrador,Canada,53.135500,-57.660400,539.606383,3.836879,NaN
14,Northwest Territories,Canada,64.825500,-124.845700,34.320922,0.000000,NaN
15,Nova Scotia,Canada,44.682000,-63.744300,1824.826241,56.611702,NaN


In [35]:
# solution to lat long problem
# objective: drop the lat long from recovered dataframe
# merge in the lat long from confirmed set

latlong_country = dataset.groupby(['province', 'country', 'lat', 'long'], as_index=False).mean()
latlong_country = latlong_country[['province', 'country', 'lat', 'long']]
latlong_country

,province,country,lat,long
0,Afghanistan,Afghanistan,33.939110,67.709953
1,Albania,Albania,41.153300,20.168300
2,Alberta,Canada,53.933300,-116.576500
3,Algeria,Algeria,28.033900,1.659600
4,Andorra,Andorra,42.506300,1.521800
...,...,...,...,...
278,Yukon,Canada,64.282300,-135.000000
279,Yunnan,China,24.974000,101.487000
280,Zambia,Zambia,-13.133897,27.849332
281,Zhejiang,China,29.183200,120.093400


In [ ]:
#Make changes to the loop to address the Canada and Lat Long problem.


In [ ]:
# solution to lat long problem
# objective: drop the lat long from recovered dataframe
# merge in the lat long from confirmed set

latlong_country=dataset.groupby(['province', 'country', 'lat', 'long'],as_index=False).mean()
latlong_country = latlong_country[['province', 'country', 'lat', 'long']]
latlong_country

In [34]:
# list of countries without province data
wo_province = ['Canada']

## For each data file,
# enumerate creates a tuple with a counter that starts from 0 and the elements of the list, e.g. (0, URL1), (1, URL2), etc.
for i, url in enumerate(COVID_DATA_URLS):

    # Load the file into memory
    df, filename = load_df_from_url(url)

    # rename columns and make them lowercase
    df.rename(columns={'Province/State': 'Province', 'Country/Region':'Country'}, inplace=True)
    df.columns = [c.lower() for c in df.columns]

    ## NEW CODES TO ADDRESS ISSUES
    # handle those countries that don't report Recovered by provinces
    df_wo_province = df[df['country'].isin(wo_province)]
    df_wo_province_group = df_wo_province.groupby(['country']).sum().reset_index()
    # print(df_wo_province_group)

    # ~ means NOT in pandas
    df = df[~df['country'].isin(wo_province)] # this will remove countries in wo_province
    df = pd.concat([df, df_wo_province_group])
    # print(df)

    ## BACK TO ORIGINAL CODES
    # fill missing provinces/states with the country/region name
    df['province'].fillna(df['country'], inplace=True)
    # print(df)

    # Create a pivot table with 'Province/State', 'Country/Region', 'Lat', 'Long' groups
    df = df.groupby(['province','country', 'lat', 'long']).sum().reset_index()
    print(df)

    # split the filename to get the keywords we want (confirmed, deaths, recovered)
    value_name = filename.split('_')[-2]

#     # NEW CODES to handle lat long problem
#     # 1. drop lat long columns from recovered
#     # 2. make new dataframe latlong_country from master dataset (confirmed) by grouping the 4 variables
#     # 3. keep only the 4 variables in the latlong_country
#     # 4. merge lat long back into recovered set
    if value_name == 'recovered':
      df.drop(columns=['lat', 'long'], inplace=True)
      latlong_country = dataset.groupby(['province', 'country', 'lat', 'long'], as_index=False).mean()
      latlong_country = latlong_country[['province', 'country', 'lat', 'long']]
      df = df.merge(latlong_country, how='left', on=['province', 'country'])

      # print(df)

    # pivot the dataframe into a panel format.
    df = df.melt(id_vars=['province','country', 'lat', 'long'], var_name='date',value_name=value_name)

    # Merge the dataframe into the main dataset
    if i==0:
        dataset = df
        # print(list(df))
    else:
        dataset = pd.merge(dataset, df, how='left', on=['province','country', 'date', 'lat', 'long'])

# convert date column to datetime
dataset['date'] = pd.to_datetime(dataset['date'])

# sort dataset  
dataset.sort_values(['province','country', 'date'], inplace=True)

# ## no longer needed with new fix: 7 Aug 2021
# to address different lat long for same province or country problem
# use fillna to fill recovered values to the same record as confirmed, deaths
# dropna to drop the recovered record
#dataset.recovered.fillna(method='backfill', inplace=True)
#dataset.dropna(subset=['confirmed', 'deaths'],how='any', inplace=True)

# reset index
dataset.reset_index(inplace=True, drop=True)

Using saved file time_series_covid19_confirmed_global.csv on colab hosted runtime
        province      country        lat        long  1/22/20  1/23/20  1/24/20  ...  8/1/21  8/2/21  8/3/21  8/4/21  8/5/21  8/6/21  8/7/21
0    Afghanistan  Afghanistan  33.939110   67.709953        0        0        0  ...  146523  147985  148572  148933  149361  149810  149810
1        Albania      Albania  41.153300   20.168300        0        0        0  ...  133121  133146  133211  133310  133442  133591  133730
2        Algeria      Algeria  28.033900    1.659600        0        0        0  ...  172564  173922  175229  176724  178013  179216  180356
3        Andorra      Andorra  42.506300    1.521800        0        0        0  ...   14678   14747   14766   14797   14809   14836   14836
4         Angola       Angola -11.202700   17.873900        0        0        0  ...   42815   42970   43070   43158   43269   43487   43592
..           ...          ...        ...         ...      ...      ...  